In [ ]:
import zstandard
import json
import sys
import orjson


file_path = "../data/annas_archive_meta__aacid__isbngrp_records__20240920T194930Z--20240920T194930Z.jsonl.seekable.zst"


# with open(file_path, 'rb') as compressed_file:
#     dctx = zstandard.ZstdDecompressor()
#     with dctx.stream_reader(compressed_file) as reader:
#         reader.readline()
        # byt = reader.read()
        # record = orjson.loads(byt)


# # Replace with the actual file name (you can dynamically set the filename if needed)
# timestamp = "20230123T123456Z"  # Replace with the actual timestamp used
# filename = f"annas_archive_meta__aacid__isbngrp_records__{timestamp}--{timestamp}.jsonl"

# # Open the JSONL file and process it line by line
# with open(filename, 'rb') as file:
#     for line in file:
#         try:
#             # Deserialize the JSON line into a Python object
            
            
#             # Example: Extracting specific fields from the record
#             aacid = record.get("aacid")
#             metadata = record.get("metadata", {})
#             record_id = metadata.get("id")
#             record_content = metadata.get("record", {})
            
#             print(f"AACID: {aacid}")
#             print(f"Record ID: {record_id}")
#             print(f"Record Content: {record_content}")
#             print("-" * 50)
#         except Exception as e:
#             print(f"Error reading line: {e}")


UnsupportedOperation: 

In [42]:
import zstandard
import orjson
import csv

# Input and output file names
output_csv_filename = "output.csv"

# Open the compressed file and CSV file
with open(file_path, 'rb') as compressed_file, open(output_csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    # Initialize the Zstandard decompressor
    dctx = zstandard.ZstdDecompressor()
    with dctx.stream_reader(compressed_file) as reader:
        # Initialize a CSV writer
        csv_writer = csv.writer(csv_file, quotechar='"', quoting=csv.QUOTE_ALL)
        
        # Write the header row to the CSV
        csv_writer.writerow(["registrant_name", "agency_name", "country_name", "isbns"])
        
        # Create a buffer to read lines incrementally
        buffer = b""
        i = 0
        while True:
            i += 1

        # for _ in range(100 000):
            chunk = reader.read(8192)  # Read chunks from the decompressed stream
            if not chunk:
                break  # End of file

            buffer += chunk
            # Process lines in the buffer
            while b'\n' in buffer:
                line, buffer = buffer.split(b'\n', 1)  # Split into a line and the remaining buffer
                try:
                    # Parse the JSON object from the line
                    record = orjson.loads(line)
                    
                    # Extract relevant fields
                    # aacid = record.get("aacid")
                    metadata = record.get("metadata", {})
                    # record_id = metadata.get("id")
                    record_content = metadata.get("record", {})
                    
                    # Extract record details
                    registrant_name = record_content.get("registrant_name")
                    agency_name = record_content.get("agency_name")
                    country_name = record_content.get("country_name")
                    
                    # Flatten the ISBNs for the CSV (comma-separated list of ISBNs)
                    isbns = "; ".join(
                        f"{isbn['isbn']} ({isbn['isbn_type']})" for isbn in record_content.get("isbns", [])
                    )
                    
                    # Write the row to the CSV
                    csv_writer.writerow([registrant_name, agency_name, country_name, isbns])
                except Exception as e:
                    print(f"Error processing line: {e}")
        
        # Handle any remaining buffer (last line without newline)
        if buffer.strip():
            try:
                record = orjson.loads(buffer)
                aacid = record.get("aacid")
                metadata = record.get("metadata", {})
                record_id = metadata.get("id")
                record_content = metadata.get("record", {})
                
                registrant_name = record_content.get("registrant_name")
                agency_name = record_content.get("agency_name")
                country_name = record_content.get("country_name")
                isbns = "; ".join(
                    f"{isbn['isbn']} ({isbn['isbn_type']})" for isbn in record_content.get("isbns", [])
                )
                
                csv_writer.writerow([aacid, record_id, registrant_name, agency_name, country_name, isbns])
            except Exception as e:
                print(f"Error processing last line: {e}")
